In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_colwidth=300
import numpy as np
import csv
from datasketch import MinHash, MinHashLSH

In [3]:
# Shingle generators 
# Arguments : Message string, shingle size {in words}
# Returns : All shingles formed with k words
def shingle_generator(message, k):
    message = message.split(' ')
    item_set = []
    for i in range(0, len(message)-k):
        shingle =  ""
        for j in range(0, k):
            shingle = shingle + message[i+j] + " "
        item_set.append(shingle)
    return item_set

def jaccard_similarity(str1, str2):
    string_set1 = set(str1.split(' '))
    string_set2 = set(str2.split(' '))
    return len(string_set1.intersection(string_set2))/len(string_set2.union(string_set1))

In [4]:
# dump pkl into a pandas dataframe. 
data = pd.read_pickle('data/string_study.pkl')
data["word_count"] = data._info.apply(lambda x:len(x.split(' ')))
msg_array = np.array(data._info.copy())
class_by_len_set = set(data.word_count)

In [5]:
# For every item in the class len set, 
# query and cluster
final_cluster = {}
for word_len in class_by_len_set:
    query_str = "word_count == " + str(word_len)
    filtered_data = data.query(query_str)
    raw_msgs = filtered_data._info
    # Restricting the data for better visualization. 
    # Remove the if statement below if you want to run it on the complete Rally dataset.
    if len(np.array(raw_msgs)) < 111:
        # Generate the set of words of all raw messages
        message_set = []
        for item in np.array(raw_msgs):
            item = str(item)
    #         set_of_words = set(item.split(' '))
            set_of_words = set(shingle_generator(item, 2))
            message_set.append(set_of_words)
        # Generate MinHash instances for all messages
        m = []
        for i in range (0, len(raw_msgs)):
            m_temp = MinHash(num_perm=128)
            m.append(m_temp)

        i = 0
        for message in message_set:
            for word in message:
                m[i].update(word.encode('utf-8'))
            i = i + 1

        lsh = MinHashLSH(threshold=.35, num_perm=128)
        count = 0
        # for i in range (0, len(m)):
        for index, row in filtered_data.iterrows():
            string = "m" + str(index)
            lsh.insert(string, m[count])
            count += 1
    #     print (count)

        # Repeat the clustering proc for every message. 
        clusters = {}
        cluster_count = 1
        # For every message add the result to the cluster
        for i in range(0, len(m)):
            # check if the message belongs to any cluster
            msg_str = "m" + str(i)
            flag = 1
            for item in clusters.values():
                if msg_str in item:
                    flag = 0
                    break
            # End of check box
            if flag:
                result = lsh.query(m[i])
                clusters[cluster_count] = result
                cluster_count += 1

        final_cluster.update(clusters)


In [1]:
# The Clusters are produced are the non radial clusters as we use a method similar to the DBscan apprach 
# which is crossed with the MinHash LSH Algorithm

# Furhter implementation procedure is mentioned in the report. 
final_cluster

NameError: name 'final_cluster' is not defined